In [ ]:
!pip install librosa -q
import librosa, librosa.display 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import os
import gc
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loading

In [ ]:
cremad_path = '/content/drive/MyDrive/USML_final_project/crema_d'
cremad_files = os.listdir(cremad_path)
print(cremad_files[0:5])

emotions_lst = []
path_lst = []

for i in cremad_files:
    path_lst.append(cremad_path + '/'+i)
    emotion_val=i.split('_')

    if emotion_val[2] == 'SAD':
        emotions_lst.append('sad')
    
    elif emotion_val[2] == 'ANG':
        emotions_lst.append('angry')
    
    elif emotion_val[2] == 'DIS':
        emotions_lst.append('disgust')
    
    elif emotion_val[2] == 'FEA':
        emotions_lst.append('fear')
    
    elif emotion_val[2] == 'HAP':
        emotions_lst.append('happy')
    
    elif emotion_val[2] == 'NEU':
        emotions_lst.append('neutral')
    
    else:
        emotions_lst.append('Unknown')

cremad = pd.DataFrame({'Path': path_lst, 'Emotion': emotions_lst})
cremad.head()

['1079_WSI_HAP_XX.wav', '1079_TSI_ANG_XX.wav', '1079_TSI_SAD_XX.wav', '1079_WSI_DIS_XX.wav', '1079_WSI_FEA_XX.wav']


,Path,Emotion
0,/content/drive/MyDrive/USML_final_project/crem...,happy
1,/content/drive/MyDrive/USML_final_project/crem...,angry
2,/content/drive/MyDrive/USML_final_project/crem...,sad
3,/content/drive/MyDrive/USML_final_project/crem...,disgust
4,/content/drive/MyDrive/USML_final_project/crem...,fear


In [ ]:
tess_path = '/content/drive/MyDrive/USML_final_project/TESS'
tess_directory_list = os.listdir(tess_path)

file_emotion = []
file_path = []
#print(tess_path)
for dir in tess_directory_list[0:3]:
    #print(dir)
    directories = os.listdir(tess_path + '/' + dir)
    #print(directories)

    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')
        if len(part) == 3:
          part = part[2]
          if part=='ps':
            file_emotion.append('surprise')
          else:
            file_emotion.append(part)
        file_path.append(tess_path + '/' + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotion'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Tess_df = pd.concat([path_df, emotion_df], axis=1)
Tess_df.head()

,Path,Emotion
0,/content/drive/MyDrive/USML_final_project/TESS...,disgust
1,/content/drive/MyDrive/USML_final_project/TESS...,disgust
2,/content/drive/MyDrive/USML_final_project/TESS...,disgust
3,/content/drive/MyDrive/USML_final_project/TESS...,disgust
4,/content/drive/MyDrive/USML_final_project/TESS...,disgust


In [ ]:
savee_path = '/content/drive/MyDrive/USML_final_project/SAVEE'
savee_directory_list = os.listdir(savee_path)

file_emotion = []
file_path = []

for file in savee_directory_list:
    file_path.append(savee_path + '/' + file)
    part = file.split('_')[1]
    ele = part[:-6]
    if ele=='a':
        file_emotion.append('angry')
    elif ele=='d':
        file_emotion.append('disgust')
    elif ele=='f':
        file_emotion.append('fear')
    elif ele=='h':
        file_emotion.append('happy')
    elif ele=='n':
        file_emotion.append('neutral')
    elif ele=='sa':
        file_emotion.append('sad')
    else:
        file_emotion.append('surprise')
        
# dataframe for path of files.
savee_df = pd.DataFrame({'Path': file_path, 'Emotion': file_emotion})
savee_df.head()

,Path,Emotion
0,/content/drive/MyDrive/USML_final_project/SAVE...,angry
1,/content/drive/MyDrive/USML_final_project/SAVE...,angry
2,/content/drive/MyDrive/USML_final_project/SAVE...,angry
3,/content/drive/MyDrive/USML_final_project/SAVE...,angry
4,/content/drive/MyDrive/USML_final_project/SAVE...,angry


In [ ]:
one_df = pd.concat([cremad, Tess_df])
one_df = pd.concat([one_df, savee_df])

In [ ]:
gc.collect()

0

# Feature Extraction

In [ ]:
%%time
SAMPLE_RATE = 22050
TRACK_DURATION = 2 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
num_mfcc=13
n_fft=2048
hop_length=512
num_segments=5

samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

data = {'mfcc': [],
        'label': []}
for i in range(one_df.shape[0]):
  if round(librosa.get_duration(filename=one_df.iloc[i,0]),1) >= 2:
    if i%1000 == 0:
      print(i)
    signal, sample_rate = librosa.load(one_df.iloc[i, 0], sr=SAMPLE_RATE)
    lst = []
    for d in range(num_segments):
      start = samples_per_segment * d
      finish = start + samples_per_segment

      # extract mfcc
      mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
      mfcc = mfcc.T
      
      if len(mfcc) == num_mfcc_vectors_per_segment:
        #print(np.array(mfcc.tolist()).shape)
        lst.append(mfcc)

        
    #print(np.array(lst).shape)
    data['mfcc'].append(np.array(lst))
    data['label'].append(one_df.iloc[i, 1])

      # store only mfcc feature with expected number of vectors

0
1000
2000
3000
4000
5000
6000
7000
8000
CPU times: user 14min 19s, sys: 16min 56s, total: 31min 15s
Wall time: 38min 15s


In [ ]:
l = []
for i in range(len(data['mfcc'])):
  if data['mfcc'][i].shape[0] != 5:
    l.append(i)
    print(i)

16
95
98
217
425
497
501
599
620
672
682
1015
1056
1100
1342
1421
1501
1563
1579
1685
1804
1904
1910
1917
2035
2054
2065
2219
2235
2253
2261
2338
2391
2405
2422
2423
2445
2538
2548
2599
2616
2626
2650
2664
2670
2725
2735
2848
2935
2971
2978
3004
3132
3136
3153
3199
3279
3353
3422
3462
3466
3493
3547
3558
3562
3648
3742
3771
3790
3792
3864
4009
4092
4159
4164
4216
4424
4451
4516
4553
4565
4658
4723
4727
4765
4791
4799
4984
5023
5052
5054
5274
5416
5639
5887
5923
5952
5953
6026
6177
6183
6185
6196
6205
6216
6235
6236
6287
6876
6878
6879
6884
6890
6894
6898
6903
6910
6920
6926
6927
6930
6949
6968
6994
6997
7005
7007


In [ ]:
lst1 = []
labels1 = []

for i in range(len(data['mfcc'])):
  if i not in l:
    lst1.append(data['mfcc'][i])
    labels1.append(data['label'][i])

In [ ]:
np.array(lst1).shape

(7553, 5, 18, 13)

# SAving the vectors in .npy format

In [ ]:
np.save('/content/drive/MyDrive/USML_final_project/Final/data_3d_final.npy',np.array(lst1))
np.save('/content/drive/MyDrive/USML_final_project/Final/labels_final.npy',np.array(labels1))